# Setup and Dependencies

In [ ]:
!pip install langchain-ollama torch transformers

In [2]:
import pandas as pd
import numpy as np
import os

import torch
from transformers import AutoTokenizer, AutoModel

from langchain_ollama import ChatOllama

c:\Users\Kraithep\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Embedding Model

In [3]:
model_name = "BAAI/bge-m3"
print(f"Loading embedding model: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Model loaded on device:", device)

Loading embedding model: BAAI/bge-m3
Model loaded on device: cpu


# Retrieval Fn.

In [4]:
def retrieve_top_chunks(query, top_k=5):
    # Embed the query
    inputs = tokenizer([query], return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        output = model(**inputs)
    q_emb = output.last_hidden_state[:, 0, :]
    q_emb = torch.nn.functional.normalize(q_emb, p=2, dim=1).cpu().numpy()[0]
    
    # Load embeddings CSV
    df = pd.read_csv("embeddings.csv")
    # Extract embedding columns (all except the first column)
    emb_cols = [col for col in df.columns if col.startswith("emb_")]
    chunk_embs = df[emb_cols].values  # shape: (num_chunks, num_dims)
    
    # Cosine similarity is just dot product (vectors are normalized)
    sims = chunk_embs.dot(q_emb)
    # Get indices of top K highest similarities
    top_indices = np.argpartition(sims, -top_k)[-top_k:]
    # Sort indices by similarity descending
    top_indices = top_indices[np.argsort(sims[top_indices])[::-1]]
    
    top_results = []
    for idx in top_indices:
        chunk_file = df.iloc[idx]["chunk_name"]
        # Read the chunk text file from the chunks/ folder
        file_path = os.path.join("chunks", chunk_file)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read().strip()
        except Exception as e:
            text = ""
        top_results.append((text, sims[idx], chunk_file))
    return top_results


# LLM Fn.

In [8]:
def build_prompt(chunks, question):
    """Builds a prompt using the retrieved chunks and the user's question."""
    context = "\n\n".join(chunks)
    prompt = (
        "You are a helpful assistant for document question answering.\n"
        "Use the following context to answer the question.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\nAnswer:"
    )
    return prompt

# Initialize the local LLM (make sure your Ollama has llama3.1:8b available)
llm = ChatOllama(model="llama3.1:8b", temperature=0)


# QA Showcases

Wisesight คืออะไร

In [9]:
sample_query = "Wisesight คืออะไร"
top_chunks = retrieve_top_chunks(sample_query, top_k=5)
print(f"Top chunks for query: \"{sample_query}\"")
for i, (chunk, sim, fname) in enumerate(top_chunks, start=1):
    print(f"Chunk {i} [{fname}, Similarity={sim:.3f}]: {chunk[:75]}...")

# Retrieve text for the prompt from top chunks
retrieved_texts = [chunk for chunk, sim, fname in top_chunks]
prompt_text = build_prompt(retrieved_texts, sample_query)

response = llm.invoke(prompt_text)
answer = response.content if hasattr(response, "content") else str(response)
print("\n\n")
print("Final Answer from LLM:\n")
print(answer)


Top chunks for query: "Wisesight คืออะไร"
Chunk 1 [wisesight.com_about-us_chunk1.txt, Similarity=0.620]: wisesight platform  service community  news about us careers  contact us  b...
Chunk 2 [wisesight.com_trend-24hours_chunk1.txt, Similarity=0.548]: wisesight platform service  community  news about us  careers  contact us  ...
Chunk 3 [thailand.zocialawards.com_2022_chunk2.txt, Similarity=0.542]: เชิดชผใช้โซเซียลมีเดียยอดเยี่ยมในสาขาต่ ไโดยได้พัฒนา wisesight meiric" เพื่...
Chunk 4 [wisesight.com_monitoring_chunk1.txt, Similarity=0.536]: wisesight platform  service  community news  about us  careers  contact us ...
Chunk 5 [wisesight.com_research_chunk1.txt, Similarity=0.535]: wisesight platform  service community news about us careers  contact us  be...



Final Answer from LLM:

Wisesight เป็นแพลตฟอร์มบริการที่ช่วยให้สามารถจัดการโซเชียลมีเดียได้อย่างมีประสิทธิภาพ และมีเครื่องมือต่างๆ เช่น วัดผลการดัดสิน (Mentor Metric), ดัชนีชีวัด (Wisesigi) และการป้องกันความไม่พึงพอใจของลูกค้า (Cr

Kirin คืออะไร

In [10]:
sample_query = "Kirin คืออะไร"
top_chunks = retrieve_top_chunks(sample_query, top_k=5)
print(f"Top chunks for query: \"{sample_query}\"")
for i, (chunk, sim, fname) in enumerate(top_chunks, start=1):
    print(f"Chunk {i} [{fname}, Similarity={sim:.3f}]: {chunk[:75]}...")

# Retrieve text for the prompt from top chunks
retrieved_texts = [chunk for chunk, sim, fname in top_chunks]
prompt_text = build_prompt(retrieved_texts, sample_query)

response = llm.invoke(prompt_text)
answer = response.content if hasattr(response, "content") else str(response)
print("\n\n")
print("Final Answer from LLM:\n")
print(answer)


Top chunks for query: "Kirin คืออะไร"
Chunk 1 [onlinemedia.idea2mobile.com_chunk5.txt, Similarity=0.485]: Package SMEs ก็ใช้งาน KIRN A.I. ได้ เริ่มต้น 3,000 บาท ต่อเดือน (รายละเอียด...
Chunk 2 [onlinemedia.idea2mobile.com_chunk20.txt, Similarity=0.475]: กับการใช้งานบางส่วนของ KIRIN Engine ใครยังไม่เคยลองใช้ควรลองใช้ดูครับ Post ...
Chunk 3 [onlinemedia.idea2mobile.com_chunk4.txt, Similarity=0.464]: เข้ามาช่วยวิเคราะห์ข้อมูลจาก Social Listening ที่ทาง Wisesight ทุ่มเงินพัฒน...
Chunk 4 [onlinemedia.idea2mobile.com_chunk8.txt, Similarity=0.436]: อยู่ตรงนี้ (ด้านขวามือ) A.I. Insight KIRIN สรุป Insigt Customer Keyword แคม...
Chunk 5 [onlinemedia.idea2mobile.com_chunk2.txt, Similarity=0.435]: insight Content Marketing Data Report Contact us Search for: You are Here H...



Final Answer from LLM:

Kirin คือเครื่องมือวิเคราะห์ข้อมูลจากโซเชียลมีเดียที่พัฒนาโดย Wisesight โดยใช้เทคนิค A.I. เพื่อให้สามารถวิเคราะห์ข้อมูลได้อย่างรวดเร็วและแม่นยำ นอกจากนี้ Kirin ยังมีให้ใช้งานฟรี 7 วัน และมีราคา 3,000

ใครพัฒนาดัชนีชี้วัดเพื่อหาผู้ชนะ THAILAND ZOCIAL AWARDS 2022

In [11]:
sample_query = "ใครพัฒนาดัชนีชี้วัดเพื่อหาผู้ชนะ THAILAND ZOCIAL AWARDS 2022"
top_chunks = retrieve_top_chunks(sample_query, top_k=5)
print(f"Top chunks for query: \"{sample_query}\"")
for i, (chunk, sim, fname) in enumerate(top_chunks, start=1):
    print(f"Chunk {i} [{fname}, Similarity={sim:.3f}]: {chunk[:75]}...")

# Retrieve text for the prompt from top chunks
retrieved_texts = [chunk for chunk, sim, fname in top_chunks]
prompt_text = build_prompt(retrieved_texts, sample_query)

response = llm.invoke(prompt_text)
answer = response.content if hasattr(response, "content") else str(response)
print("\n\n")
print("Final Answer from LLM:\n")
print(answer)

Top chunks for query: "ใครพัฒนาดัชนีชี้วัดเพื่อหาผู้ชนะ THAILAND ZOCIAL AWARDS 2022"
Chunk 1 [thailand.zocialawards.com_2022_chunk1.txt, Similarity=0.598]: thailand  zocia awards ^  anniversary  live  thalland zocial awards 22 febr...
Chunk 2 [thailand.zocialawards.com_2022_chunk9.txt, Similarity=0.481]: ถือเป็นรางวัลที่มอบให้กับแบรนด์หรือเอเจนซีเพื่อเชิดชผลงานที่ใช้โซเซียลมี เด...
Chunk 3 [thailand.zocialawards.com_2022_chunk2.txt, Similarity=0.442]: เชิดชผใช้โซเซียลมีเดียยอดเยี่ยมในสาขาต่ ไโดยได้พัฒนา wisesight meiric" เพื่...
Chunk 4 [wisesight.com_command-center_chunk1.txt, Similarity=0.424]: wisesight platform servi  community about careers  contact us  become a par...
Chunk 5 [thailand.zocialawards.com_2022_chunk3.txt, Similarity=0.419]: กาหรับการวัดผลบบ งผู่ไช้ไนปร โดยวัดผลใน 4 ช่อ แและ youtube โดยแบ่ง iric ที่...



Final Answer from LLM:

บริษัท วิช์ไซท์ (ประเทศไทย) จำกัด


BRAND METRIC คืออะไร

In [12]:
sample_query = "BRAND METRIC คืออะไร"
top_chunks = retrieve_top_chunks(sample_query, top_k=5)
print(f"Top chunks for query: \"{sample_query}\"")
for i, (chunk, sim, fname) in enumerate(top_chunks, start=1):
    print(f"Chunk {i} [{fname}, Similarity={sim:.3f}]: {chunk[:75]}...")

# Retrieve text for the prompt from top chunks
retrieved_texts = [chunk for chunk, sim, fname in top_chunks]
prompt_text = build_prompt(retrieved_texts, sample_query)

response = llm.invoke(prompt_text)
answer = response.content if hasattr(response, "content") else str(response)
print("\n\n")
print("Final Answer from LLM:\n")
print(answer)

Top chunks for query: "BRAND METRIC คืออะไร"
Chunk 1 [thailand.zocialawards.com_2022_chunk4.txt, Similarity=0.496]: บดคลมีผ่อเนือหาของแบรนด์และ mention สำบวนการพดถึงแบรนด์บนโซเซียลมีเดีย ประส...
Chunk 2 [thailand.zocialawards.com_2022_chunk2.txt, Similarity=0.474]: เชิดชผใช้โซเซียลมีเดียยอดเยี่ยมในสาขาต่ ไโดยได้พัฒนา wisesight meiric" เพื่...
Chunk 3 [thailand.zocialawards.com_2022_chunk5.txt, Similarity=0.458]: entertainment metric ertanmen ใช่สาหรับวัด จถึงในวงกว่าง โดยมีปัจจัยในการวั...
Chunk 4 [thailand.zocialawards.com_2022_chunk3.txt, Similarity=0.449]: กาหรับการวัดผลบบ งผู่ไช้ไนปร โดยวัดผลใน 4 ช่อ แและ youtube โดยแบ่ง iric ที่...
Chunk 5 [thailand.zocialawards.com_2022_chunk6.txt, Similarity=0.447]: และในส่วนของกล่มรางวัลผผลิตร การออนไลน์ (online channel) จะใช่วิธีการวัดผลท...



Final Answer from LLM:

 BRAND METRIC คือระบบการวัดผลและประเมินประสิทธิภาพของแบรนด์บนโซเชียลมีเดีย โดยใช้หลักการวัดผลที่ครอบคลุมหลายด้าน เช่น ประสิทธิภาพเชิงปริมาณ (quantity) และประสิทธิภาพเชิงคุณภาพ (q

ENTERTAINMENT METRIC คืออะไร

In [13]:
sample_query = "ENTERTAINMENT METRIC คืออะไร"
top_chunks = retrieve_top_chunks(sample_query, top_k=5)
print(f"Top chunks for query: \"{sample_query}\"")
for i, (chunk, sim, fname) in enumerate(top_chunks, start=1):
    print(f"Chunk {i} [{fname}, Similarity={sim:.3f}]: {chunk[:75]}...")

# Retrieve text for the prompt from top chunks
retrieved_texts = [chunk for chunk, sim, fname in top_chunks]
prompt_text = build_prompt(retrieved_texts, sample_query)

response = llm.invoke(prompt_text)
answer = response.content if hasattr(response, "content") else str(response)
print("\n\n")
print("Final Answer from LLM:\n")
print(answer)

Top chunks for query: "ENTERTAINMENT METRIC คืออะไร"
Chunk 1 [thailand.zocialawards.com_2022_chunk5.txt, Similarity=0.600]: entertainment metric ertanmen ใช่สาหรับวัด จถึงในวงกว่าง โดยมีปัจจัยในการวั...
Chunk 2 [thailand.zocialawards.com_2022_chunk6.txt, Similarity=0.476]: และในส่วนของกล่มรางวัลผผลิตร การออนไลน์ (online channel) จะใช่วิธีการวัดผลท...
Chunk 3 [thailand.zocialawards.com_2022_chunk3.txt, Similarity=0.459]: กาหรับการวัดผลบบ งผู่ไช้ไนปร โดยวัดผลใน 4 ช่อ แและ youtube โดยแบ่ง iric ที่...
Chunk 4 [onlinemedia.idea2mobile.com_chunk6.txt, Similarity=0.450]: วัดจาก Message ที่เข้ามาในระบบเปรียบเทียบสัดส่วน 4.Engagement การมีส่วนร่วม...
Chunk 5 [thailand.zocialawards.com_2022_chunk2.txt, Similarity=0.449]: เชิดชผใช้โซเซียลมีเดียยอดเยี่ยมในสาขาต่ ไโดยได้พัฒนา wisesight meiric" เพื่...



Final Answer from LLM:

ENTERTAINMENT METRIC เป็นระบบการวัดผลในการประเมินความสำเร็จของผลงานในวงการบันเทิง โดยมีปัจจัยในการวัดผลที่เกี่ยวข้องกับกลุ่มเป้าหมาย, ระดับการกระจายเนื้อหาผ่านโซเชียลมีเดีย,

เราจะ maximize customer satisfaction ได้ยังไง

In [14]:
sample_query = "เราจะ maximize customer satisfaction ได้ยังไง"
top_chunks = retrieve_top_chunks(sample_query, top_k=5)
print(f"Top chunks for query: \"{sample_query}\"")
for i, (chunk, sim, fname) in enumerate(top_chunks, start=1):
    print(f"Chunk {i} [{fname}, Similarity={sim:.3f}]: {chunk[:75]}...")

# Retrieve text for the prompt from top chunks
retrieved_texts = [chunk for chunk, sim, fname in top_chunks]
prompt_text = build_prompt(retrieved_texts, sample_query)

response = llm.invoke(prompt_text)
answer = response.content if hasattr(response, "content") else str(response)
print("\n\n")
print("Final Answer from LLM:\n")
print(answer)

Top chunks for query: "เราจะ maximize customer satisfaction ได้ยังไง"
Chunk 1 [wisesight.com_about-us_chunk2.txt, Similarity=0.515]: have been growing strongly. over 14 years  experience, we have collected  t...
Chunk 2 [wisesight.com_warroom_chunk2.txt, Similarity=0.510]: social channel  warroom helns you maximize customer satisfaction as the pla...
Chunk 3 [wisesight.com_consulting_chunk7.txt, Similarity=0.500]: gain the riaht un to date and useful consumer insiahts instantly in time fo...
Chunk 4 [wisesight.com_warroom_chunk1.txt, Similarity=0.499]: wisesight plateopm  sepvicf  coumumtv  mfws  about rapfeps  routart us เ06เ...
Chunk 5 [wisesight.com_about-us_chunk5.txt, Similarity=0.499]: more about our cookie policy  allow us...



Final Answer from LLM:

เราสามารถ maximize customer satisfaction ได้โดยใช้ Social Channel Warroom ซึ่งเป็นแพลตฟอร์มที่ช่วยให้คุณสามารถจัดการและตรวจสอบการกล่าวถึงของลูกค้าในโซเชียลมีเดียได้อย่างครอบคลุม และยังช่วยให้คุณสามารถตอบสนองความต้องการของลูกค้าอย่